# Práctica 1: Sergio Soler Rocha
## Objetivo
En el presente trabajo práctico se abordará el diseño e implementación de una base de datos clave-valor utilizando Redis para gestionar un servicio de microblogging similar a Twitter. El propósito principal es adquirir familiaridad con el uso de bases de datos clave-valor, como Redis, y verificar que sus estructuras permiten la construcción efectiva de modelos de datos para diversas aplicaciones. El enfoque del proyecto será crear un "clon" de Twitter que sea estructuralmente simple, funcione eficientemente y pueda ser escalable para ser distribuido entre múltiples servidores con facilidad. Se empleará la librería redis-py de Python debido a su gran funcionalidad para los objetivos del proyecto.

In [11]:
# Instalación de Redis
!pip install redis

In [23]:
# Importar la libreía Redis
import redis

# Conexión a la base de datos local de Redis
redis_db=redis.Redis(host='127.0.0.1',port=6379, password='')

# Limpieza de la base de datos Redis
redis_db.flushdb()

True

## Parte 1: Diseño de la base de datos

- La función **nuevo_usuario** realiza la tarea de agregar un nuevo usuario a Redis, generando un identificador incremental, creando una clave única para el usuario, almacenando el nombre del usuario y estableciendo la relación entre el nombre y el identificador en una estructura de datos adicional.

In [24]:
def nuevo_usuario(nombre_usuario):
    # Obtener un identificador incremental para el nuevo usuario
    id_usuario = redis_db.incr("user_id_counter")
    
    # Crear la clave para el nuevo usuario
    clave_usuario = f"user:{id_usuario}"
    
    # Almacenar el nombre del usuario en la clave correspondiente
    redis_db.hset(clave_usuario, "nombre", nombre_usuario)
    
    # Almacenar la relación usuario-identificador en la estructura de datos 'users'
    redis_db.hset("users", nombre_usuario, id_usuario)

    print(f"Nuevo usuario creado: {clave_usuario} - Nombre: {nombre_usuario}")

- La función **nuevo_follower** tiene como objetivo registrar la acción de que un usuario (usuario_follower) ha comenzado a seguir a otro usuario (usuario_original) en una base de datos Redis. Recibe el nombre del usuario original que será seguido (usuario_original), el nombre del usuario seguidor (usuario_follower) y un timestamp que indica el momento en que se realizó la acción de seguir. Utiliza el comando HGET de Redis para obtener los identificadores de los usuarios original y seguidor a partir de sus nombres almacenados en un hash llamado "users". Comprueba si ambos usuarios existen en la base de datos. Si alguno de ellos no existe, imprime un mensaje de error.  Si ambos usuarios existen, la función crea una clave única para almacenar los seguidores del usuario original. Luego, utiliza el comando ZADD de Redis para agregar al conjunto ordenado (sorted set) correspondiente al usuario original, el nombre del seguidor junto con una puntuación dada, que en este caso es el timestamp. Esto permite ordenar los seguidores por el momento en que comenzaron a seguir al usuario original.

In [25]:
import time

def nuevo_follower(usuario_original, usuario_follower, timestamp):
    # Obtener el identificador del usuario original a partir de su nombre utilizando el comando HGET.
    id_usuario_original = redis_db.hget("users", usuario_original)
    
    # Obtener el identificador del usuario seguidor a partir de su nombre utilizando el comando HGET.
    id_usuario_follower = redis_db.hget("users", usuario_follower)

    # Verificar si ambos usuarios existen en la base de datos. Si alguno de ellos no existe, la función imprime un mensaje de error.
    if id_usuario_original and id_usuario_follower:
        # Crear una clave única para almacenar los seguidores del usuario original utilizando el identificador del usuario original.
        clave_followers = f"followers:{id_usuario_original}"
        
        # Utilizar el comando ZADD para agregar al conjunto ordenado (sorted set) clave_followers al usuario seguidor junto con una puntuación dada, que en este caso es el timestamp. Esto permite ordenar los seguidores por el momento en que comenzaron a seguir al usuario original.
        redis_db.zadd(clave_followers, {usuario_follower: timestamp})
        print(f"{usuario_follower} ha comenzado a seguir a {usuario_original} en {datetime.utcfromtimestamp(int(timestamp)).strftime('%Y-%m-%d %H:%M:%S')}")
    else:
        print("Error: Usuario no encontrado en la base de datos.")


- El siguiente código define la función **nuevo_following**, que tiene como propósito registrar la acción de que un usuario (usuario_following) ha comenzado a seguir a otro usuario (usuario_original) en una base de datos Redis. Registra la relación de "following" entre dos usuarios en Redis, almacenando esta información en una estructura específica si ambos usuarios existen en la base de datos. Recibe el nombre del usuario original que sigue a otro usuario (usuario_original), el nombre del usuario que está siendo seguido (usuario_following) y un timestamp que indica el momento en que se realizó la acción de seguir. Utiliza el comando HGET de Redis para obtener los identificadores de los usuarios original y el que está siendo seguido a partir de sus nombres almacenados en un hash llamado "users". Comprueba si ambos usuarios existen en la base de datos. Si alguno de ellos no existe, imprime un mensaje de error. Si ambos usuarios existen, la función crea una clave única para almacenar a quiénes sigue el usuario original. Luego, utiliza el comando ZADD de Redis para agregar al conjunto ordenado (sorted set) correspondiente al usuario original, el nombre del usuario que está siendo seguido junto con una puntuación dada, que en este caso es el timestamp. Esto permite ordenar los usuarios que el usuario original está siguiendo por el momento en que comenzaron a seguirlos.

In [26]:
def nuevo_following(usuario_original, usuario_following, timestamp):
    # Obtiene el identificador del usuario original a partir de su nombre utilizando el comando HGET de Redis
    id_usuario_original = redis_db.hget("users", usuario_original)
    
    #  Obtener el identificador del usuario que está siguiendo a otro a partir de su nombre
    id_usuario_following = redis_db.hget("users", usuario_following)
    
    # Verificar si ambos usuarios existen en la base de datos
    if id_usuario_original and id_usuario_following:
        # Crea una clave única para almacenar a quiénes sigue el usuario original utilizando el identificador del usuario original
        clave_following = f"following:{id_usuario_original}"
        
        # Utilizar el comando ZADD de Redis para agregar al conjunto ordenado (sorted set) clave_following al usuario que está siendo seguido junto con una puntuación dada, que en este caso es el timestamp. Esto permite ordenar los usuarios que el usuario original está siguiendo por el momento en que comenzaron a seguirlos.
        redis_db.zadd(clave_following, {usuario_following: timestamp})
    else:
        print("Error: Usuario no encontrado en la base de datos.")


- La función **seguir** se encarga de establecer una relación de seguimiento entre dos usuarios en un sistema. La función utiliza dos funciones auxiliares definidas en los pasos anteriores: **nuevo_follower** y **nuevo_following**

In [27]:
def seguir(usuario_original, usuario_a_seguir, timestamp):
    # Llamar a la función auxiliar para actualizar followers
    nuevo_follower(usuario_original, usuario_a_seguir, timestamp)
    
    # Llamar a la función auxiliar para actualizar followings
    nuevo_following(usuario_a_seguir, usuario_original, timestamp)




- La función **nuevo_post** realiza la tarea de agregar un nuevo post en el sistema, almacenando información asociada como el usuario creador, el timestamp y el cuerpo del mensaje. Además, actualiza las listas de posts del usuario creador y de sus seguidores. Recibe el nombre del usuario creador del post (usuario_creador), el cuerpo del mensaje del post (cuerpo_mensaje), y un timestamp que indica el momento en que se creó el post. Utiliza el comando INCR de Redis para obtener un identificador único e incremental para el nuevo post, almacenado en una clave denominada "post_id_counter". Utiliza el identificador del post para crear una clave única en Redis, concatenando "post:" con el identificador del post.  Utiliza el comando HSET de Redis para almacenar el nombre del usuario creador, el timestamp y el cuerpo del mensaje en la clave del post. Obtiene el identificador del usuario creador a partir de su nombre y crea una clave única para la lista de posts del usuario. Luego, utiliza el comando ZADD de Redis para agregar el identificador del nuevo post a esta lista, ordenado por el timestamp. Obtiene y almacena los identificadores de posts de los seguidores del usuario creador. Itera sobre cada seguidor y almacena el identificador del nuevo post en las listas de posts de los seguidores, también ordenado por el timestamp. 

In [28]:
from datetime import datetime

def nuevo_post(usuario_creador, cuerpo_mensaje, timestamp):
    # Obtener un identificador incremental para el nuevo post
    id_post = redis_db.incr("post_id_counter")
    
    # Crear la clave para el nuevo post
    clave_post = f"post:{id_post}"
    
    
    # Utilizar el comando HSET de Redis para almacenar el nombre del usuario creador en la clave del post
    redis_db.hset(clave_post, "usuario_creador", usuario_creador)
    
    # Almacenar el timestamp del post en la misma clave.
    redis_db.hset(clave_post, "timestamp", timestamp)
    
    # Almacenar el cuerpo del mensaje del post en la misma clave.
    redis_db.hset(clave_post, "cuerpo_mensaje", cuerpo_mensaje)
    
    
    # Obtener el identificador del usuario creador a partir de su nombre.
    id_usuario_creador = redis_db.hget("users", usuario_creador)
    
    # Crear una clave única para la lista de posts del usuario creador.
    clave_posts_usuario = f"posts:{id_usuario_creador}"
    
    #Utiliza el comando ZADD de Redis para agregar el identificador del nuevo post a la lista de posts del usuario creador, ordenado por el timestamp.
    redis_db.zadd(clave_posts_usuario, {str(id_post): timestamp})
    
    
    # Obtener y almacenar los identificadores de posts de los seguidores (followings)
    followers = redis_db.zrange(f"followers:{id_usuario_creador}", 0, -1)
    
    # Se itera sobre cada seguidor y se almacena el identificador del nuevo post en sus listas de posts, también ordenado por timestamp
    for follower_id in followers:
        clave_posts_follower = f"posts:{follower_id}"
        redis_db.zadd(clave_posts_follower, {str(id_post): timestamp})
    
    print(f"Nuevo post creado: {clave_post} - Usuario: {usuario_creador} - Timestamp: {timestamp}")

## Parte 2: Conjunto de datos
Una vez que hemos establecido las funciones necesarias para implementar nuestra base de datos, procederemos a poblarla con un conjunto de datos predefinido. Estos datos están disponibles en dos archivos CSV (Valores Separados por Comas) llamados "twitter_sample.csv" y "relations.csv". Para eso haremos uso de las siguientes funciones:

- La función **crear_usuarios_desde_csv** lee el archivo CSV que contiene información de usuarios y llama a la función **nuevo_usuario** para cada usuario nuevo encontrado en el archivo. Recibe el nombre del archivo CSV que contiene los datos de los usuarios a crear (archivo_csv). Se crea un conjunto vacío llamado usuarios_creados para almacenar los nombres de usuario que ya han sido creados para evitar la duplicación. Abre el archivo CSV en modo de lectura y utiliza csv.DictReader para leer el archivo y crear un diccionario por cada fila, donde las claves del diccionario son los nombres de las columnas del archivo CSV. Itera sobre cada fila del archivo CSV. Obtiene el nombre de usuario de cada fila del CSV utilizando la clave 'User'. Verifica si el nombre de usuario ya ha sido creado previamente. Si no está en el conjunto usuarios_creados, se procede a crear el usuario utilizando la función **nuevo_usuario**. Agrega el nombre de usuario al conjunto usuarios_creados para evitar su creación repetida.

- La función **establecer_relaciones_desde_csv** lee el archivo CSV que contiene información sobre relaciones de seguimiento y llama a la función **seguir** para establecer estas relaciones. Recibe el nombre del archivo CSV que contiene los datos de las relaciones de seguimiento (archivo_csv). Abre el archivo CSV en modo de lectura y utiliza csv.DictReader para leer el archivo y crear un diccionario por cada fila, donde las claves del diccionario son los nombres de las columnas del archivo CSV. Itera sobre cada fila del archivo CSV. Extrae la fecha y hora del seguimiento de cada fila del CSV y la convierte en un objeto de tipo datetime utilizando strptime con el formato especificado ("%d %b %Y %H:%M:%S"). Luego, convierte esta fecha y hora en un timestamp Unix utilizando el método timestamp(). Utiliza la función **seguir** para establecer la relación de seguimiento entre el usuario especificado en la columna 'User' y el usuario al que sigue especificado en la columna 'Follows', utilizando el timestamp calculado en el paso anterior como indicador de cuándo se realizó el seguimiento.

- La función **agregar_tweets_desde_csv** lee un archivo CSV que contiene información de tweets y llama a la función **nuevo_post** para cada tweet encontrado en el archivo. Recibe el nombre del archivo CSV que contiene los datos de los tweets (archivo_csv). Abre el archivo CSV en modo de lectura y utiliza csv.DictReader para leer el archivo y crear un diccionario por cada fila, donde las claves del diccionario son los nombres de las columnas del archivo CSV. Itera sobre cada fila del archivo CSV. Extrae la fecha y hora del tweet de cada fila del CSV y la convierte en un objeto de tipo datetime utilizando strptime con el formato especificado ("%d %b %Y %H:%M:%S"). Luego, convierte esta fecha y hora en un timestamp Unix utilizando el método timestamp().  Utiliza la función **nuevo_post** para crear un nuevo tweet, pasando como parámetros el usuario que creó el tweet especificado en la columna 'User', el contenido del tweet especificado en la columna 'Tweet_Content' y el timestamp calculado en el paso anterior como indicador de cuándo se publicó el tweet.

In [29]:
import csv
from datetime import datetime

# Función para crear usuarios desde el archivo CSV
def crear_usuarios_desde_csv(archivo_csv):
    # Conjunto para almacenar los nombres de usuario ya creados
    usuarios_creados = set()
    
    with open(archivo_csv, 'r', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            nombre_usuario = row['User']
            
            # Verificar si el usuario ya fue creado
            if nombre_usuario not in usuarios_creados:
                nuevo_usuario(nombre_usuario)
                usuarios_creados.add(nombre_usuario)

# Función para establecer relaciones de seguimiento desde el archivo CSV
def establecer_relaciones_desde_csv(archivo_csv):
    with open(archivo_csv, 'r', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            fecha_following = datetime.strptime(row['Following_Time'], "%d %b %Y %H:%M:%S")
            timestamp_following = int(fecha_following.timestamp())
            seguir(row['User'], row['Follows'], timestamp_following)

def agregar_tweets_desde_csv(archivo_csv):
    with open(archivo_csv, 'r', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            fecha_post = datetime.strptime(row['Post_Time'], "%d %b %Y %H:%M:%S")
            timestamp_post = int(fecha_post.timestamp())
            nuevo_post(row['User'], row['Tweet_Content'], timestamp_post)

# Poblar la base de datos con usuarios
crear_usuarios_desde_csv("twitter_sample.csv")

# Establecer relaciones de seguimiento
establecer_relaciones_desde_csv("relations.csv")

# Agregar tweets
agregar_tweets_desde_csv("twitter_sample.csv")



Nuevo usuario creado: user:1 - Nombre: andyglittle
Nuevo usuario creado: user:2 - Nombre: afparron
Nuevo usuario creado: user:3 - Nombre: drshahrul80
Nuevo usuario creado: user:4 - Nombre: karin_stowell
Nuevo usuario creado: user:5 - Nombre: cathcooney
Nuevo usuario creado: user:6 - Nombre: dkalnow
Nuevo usuario creado: user:7 - Nombre: alkhalilkouma
Nuevo usuario creado: user:8 - Nombre: seers_helen
Nuevo usuario creado: user:9 - Nombre: hanyshita
Nuevo usuario creado: user:10 - Nombre: 
Nuevo usuario creado: user:11 - Nombre: roxanefeller
Nuevo usuario creado: user:12 - Nombre: animalhealthEU
Nuevo usuario creado: user:13 - Nombre: charleskod
cathcooney ha comenzado a seguir a roxanefeller en 2019-06-13 03:59:58
charleskod ha comenzado a seguir a andyglittle en 2019-07-14 08:07:29
seers_helen ha comenzado a seguir a andyglittle en 2019-07-18 07:50:48
karin_stowell ha comenzado a seguir a andyglittle en 2019-08-31 13:20:48
andyglittle ha comenzado a seguir a hanyshita en 2019-07-12 12

## Parte 3: Pruebas

- La función **obtener_followers** busca en la base de datos los seguidores de un usuario dado, si existen, imprime sus nombres y el momento en que comenzaron a seguir al usuario dado. Si el usuario no tiene seguidores o si el usuario no se encuentra en la base de datos, se imprime un mensaje de que no tiene seguidores o de que el usuario no existe respectivamente. Recibe el nombre de usuario del cual se desean obtener los seguidores (usuario). Utiliza el método hget de Redis para obtener el identificador del usuario a partir de su nombre de usuario. Si el identificador del usuario existe, significa que el usuario está registrado en la base de datos. Crea la clave necesaria para acceder a la estructura de datos que almacena los seguidores del usuario. La clave se forma concatenando "followers:" con el identificador del usuario. Utiliza el método zrange de Redis para obtener una lista de seguidores del usuario junto con sus timestamps. Si hay seguidores, imprime la información de cada seguidor, mostrando su nombre de usuario y la fecha en que comenzaron a seguir al usuario específico. Si el usuario no tiene seguidores, se imprime un mensaje indicándolo. Si el usuario no se encuentra en la base de datos, se imprime un mensaje de error indicando que el usuario no fue encontrado.

In [30]:
def obtener_followers(usuario):
    # Utilizar el método hget para obtener el identificador del usuario a partir de su nombre de usuario
    id_usuario = redis_db.hget("users", usuario)
    
    # Si el identificador existe, significa que el usuario está registrado en la base de datos
    if id_usuario:
        # Crea la clave para acceder a la estructura de datos que almacena los seguidores del usuario. La clave se forma concatenando "followers:" con el identificador del usuario
        clave_followers = f"followers:{id_usuario}"
        
        # Utiliza el método zrange para obtener una lista de seguidores del usuario junto con sus timestamps
        followers_data = redis_db.zrange(clave_followers, 0, -1, withscores=True)

        # Si hay seguidores, se imprime la información
        if followers_data:
            print(f"Seguidores de {usuario}:")
            for follower, timestamp in followers_data:
                # Decodificar el nombre de usuario si es un objeto de bytes
                follower_str = follower.decode('utf-8') if isinstance(follower, bytes) else follower
                # Formatear la fecha
                fecha_formateada = datetime.utcfromtimestamp(int(timestamp)).strftime('%Y-%m-%d %H:%M:%S')
                print(f"{follower_str} - Comenzó a seguirlo el {fecha_formateada}")
        else:
            print(f"{usuario} no tiene seguidores.")
    else:
        print(f"Usuario {usuario} no encontrado en la base de datos.")

# Ejemplo
obtener_followers("cathcooney")


Seguidores de cathcooney:
alkhalilkouma - Comenzó a seguirlo el 2019-06-25 10:20:37


- La función **obtener_following** busca en la base de datos a los usuarios que sigue un usuario dado, si existen, imprime sus nombres y el momento en que comenzaron a ser seguidos por dicho usuario. Si el usuario no tiene seguidores o si el usuario no se encuentra en la base de datos, se imprime un mensaje de que no sigue a ningún usuario o de que el usuario no existe respectivamente. Recibe el nombre de usuario del cual se desean obtener los usuarios seguidos (usuario). Utiliza el método hget de Redis para obtener el identificador del usuario a partir de su nombre de usuario. Si el identificador del usuario existe, significa que el usuario está registrado en la base de datos. Crea la clave necesaria para acceder a la estructura de datos que almacena los usuarios seguidos por el usuario. La clave se forma concatenando "following:" con el identificador del usuario. Utiliza el método zrange de Redis para obtener una lista de usuarios seguidos por el usuario junto con sus timestamps. Si hay usuarios seguidos, imprime la información de cada uno, mostrando su nombre de usuario y la fecha en que comenzaron a ser seguidos por el usuario específico. Si el usuario no sigue a ningún usuario, se imprime un mensaje indicándolo. Si el usuario no se encuentra en la base de datos, se imprime un mensaje de error indicando que el usuario no fue encontrado.

In [31]:
def obtener_following(usuario):
    # Utilizar el método hget para obtener el identificador del usuario a partir de su nombre de usuario
    id_usuario = redis_db.hget("users", usuario)

    # Si el identificador existe, significa que el usuario está registrado en la base de datos
    if id_usuario:
        # Crea la clave para acceder a la estructura de datos que almacena los usuarios seguidos por el usuario. La clave se forma concatenando "following:" con el identificador del usuario
        clave_following = f"following:{id_usuario}"
        
        # Utiliza el método zrange para obtener una lista de usuarios seguidos por el usuario junto con sus timestamps
        following_data = redis_db.zrange(clave_following, 0, -1, withscores=True)

        # Si hay usuarios seguidos, se imprime la información
        if following_data:
            print(f"Usuarios seguidos por {usuario}:")
            for following, timestamp in following_data:
                # Decodificar el nombre de usuario si es un objeto de bytes
                following_str = following.decode('utf-8') if isinstance(following, bytes) else following
                # Formatear la fecha
                fecha_formateada = datetime.utcfromtimestamp(int(timestamp)).strftime('%Y-%m-%d %H:%M:%S')
                print(f"{following_str} - Comenzó a seguirlo el {fecha_formateada}")
        else:
            print(f"{usuario} no sigue a ningún usuario.")
    else:
        print(f"Usuario {usuario} no encontrado en la base de datos.")

# Ejemplo
obtener_following("cathcooney")

Usuarios seguidos por cathcooney:
roxanefeller - Comenzó a seguirlo el 2019-06-13 03:59:58
drshahrul80 - Comenzó a seguirlo el 2019-07-01 18:56:39


- La función **obtener_timeline** se encarga de obtener y mostrar el timeline de un usuario del servicio de microblogging, que consiste en una lista de tweets de los usuarios que sigue. Esta función proporciona una manera de obtener y mostrar el timeline de un usuario, incluyendo los tweets de los usuarios que sigue, utilizando la base de datos Redis como almacenamiento. Recibe el nombre de usuario del cual se desea obtener el timeline (usuario) y un parámetro opcional tweets_propios que indica si se deben incluir los propios tweets del usuario en el timeline (por defecto es True). Utiliza el método hget de Redis para obtener el identificador del usuario a partir de su nombre de usuario. Si el identificador del usuario existe, significa que el usuario está registrado en la base de datos. Utiliza el método zrange de Redis para obtener una lista de identificadores de los usuarios que sigue el usuario. Crea una clave temporal para almacenar temporalmente los tweets del timeline, esta clave se forma concatenando "temporal_timeline:" con el identificador del usuario. Itera sobre la lista de identificadores de los usuarios que sigue el usuario. Para cada usuario seguido, utiliza el método zunionstore de Redis para obtener los tweets del usuario seguido y almacenarlos en la clave temporal.  Utiliza el método zrevrange de Redis para obtener los tweets almacenados en la clave temporal, luego muestra la información de cada tweet, incluyendo el nombre del usuario creador, el contenido del tweet y la fecha de publicación. Una vez que se han mostrado los tweets del timeline, se elimina la clave temporal para limpiar los datos temporales. En lugar de utilizar una estructura de datos como una lista que se puede ordenar directamente con SORT, el código utiliza conjuntos ordenados (sorted sets) para almacenar los tweets de cada usuario. Estos conjuntos ordenados almacenan los tweets ordenados por su timestamp, lo que facilita la obtención de los tweets en orden cronológico inverso utilizando ZREVRANGE.

In [32]:
def obtener_timeline(usuario, tweets_propios=True):
    id_usuario = redis_db.hget("users", usuario)

    if id_usuario:
        # Obtener los identificadores de los usuarios seguidos
        clave_following = f"following:{id_usuario}"
        usuarios_seguidos = redis_db.zrange(clave_following, 0, -1)

        if usuarios_seguidos:
            # Crear una clave temporal para almacenar los tweets del timeline
            clave_temporal = f"temporal_timeline:{id_usuario}"

            # Recorrer los usuarios seguidos
            for usuario_seguido_id in usuarios_seguidos:
                # Obtener la clave de los tweets del usuario seguido
                clave_tweets_usuario_seguido = f"posts:{usuario_seguido_id}"

                # Obtener los tweets propios o de los usuarios seguidos
                if tweets_propios:
                    redis_db.zunionstore(clave_temporal, {clave_tweets_usuario_seguido: 1})
                else:
                    # Obtener el momento en el que se empezó a seguir al usuario
                    timestamp_seguimiento = redis_db.zscore(clave_following, usuario_seguido_id)
                    if timestamp_seguimiento:
                        redis_db.zunionstore(clave_temporal, {clave_tweets_usuario_seguido: 1}, aggregate="MAX", weights=[0, 1])
            
            # Obtener y mostrar los tweets ordenados por fecha
            tweets_data = redis_db.zrevrange(clave_temporal, 0, -1, withscores=True)
            if tweets_data:
                print(f"Timeline de {usuario}:")
                for tweet_id, timestamp in tweets_data:
                    tweet_id_str = tweet_id if isinstance(tweet_id, str) else tweet_id.decode('utf-8')
                    tweet_info = redis_db.hgetall(f"post:{tweet_id_str}")
                    if tweet_info:
                        usuario_creador = tweet_info.get(b'usuario_creador', b'').decode('utf-8')
                        cuerpo_mensaje = tweet_info.get(b'cuerpo_mensaje', b'').decode('utf-8')
                        fecha_formateada = datetime.utcfromtimestamp(int(timestamp)).strftime('%Y-%m-%d %H:%M:%S')
                        print(f"{usuario_creador} \n {cuerpo_mensaje} \n Publicado el {fecha_formateada}")
                    else:
                        print(f"Error: No se pudieron obtener detalles del tweet con ID {tweet_id_str}")

                # Eliminar la clave temporal
                redis_db.delete(clave_temporal)
            else:
                print(f"{usuario} no tiene tweets en el timeline.")
        else:
            print(f"{usuario} no sigue a ningún usuario.")
    else:
        print(f"Usuario {usuario} no encontrado en la base de datos.")


In [31]:
obtener_timeline("andyglittle")

Timeline de andyglittle:
andyglittle 
 Another spot of our #morethanmedicine bus in #bristol this week! If you need support with your cancer diagnosis call us on 0303 3000 118. #livingwellwithcancer https://t.co/eZGLz0BkXB 
 Publicado el 2019-08-30 07:55:43
andyglittle 
 For nearly 40 years @PennyBrohnUK has been a link between standard medical treatment and social prescribing, that's why we're at the second @SocialPrescrib2. https://t.co/nD6V7aG5lI #morethanmedicine #socialprescribing2019 https://t.co/c8dRWTbtx5 
 Publicado el 2019-08-27 14:07:40
andyglittle 
 @somedocs #MoreThanMedicine 
#MedEd
#FOAMed 

@dkalnow partner in crime, self appointed grammar cop, a voice of reason... 
 Publicado el 2019-08-22 07:44:36
andyglittle 
 This July we'll be sharing the stories of our Board company members through our #MorethanMedicine campaign:
https://t.co/aAO0qA35j0
Get to know the people driving the future of #animalhealth &amp; learn more about what gets our tails wagging. 
 Publicado el 201